In [3]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import random
from scipy.signal import sepfir2d
import math
%matplotlib inline

from os import path, getcwd
from glob import glob

code_dir = getcwd()
data_dir = path.expanduser('~/data/workshop-content18/3-snc/data/')

ais_pathnames = glob(data_dir + '*.txt')
ais_basenames = [path.basename(pn) for pn in ais_pathnames]

delta_cur_basename = next(bn for bn in ais_basenames if 'Deltaport_Current' in bn)
delta_his_basename = next(bn for bn in ais_basenames if 'Deltaport_History' in bn)
nwest_cur_basename = next(bn for bn in ais_basenames if 'NewWestminster_Current' in bn)
nwest_his_basename = next(bn for bn in ais_basenames if 'NewWestminster_History' in bn)

max_rows=int(3e6)
delta_cur = pd.read_csv(
    data_dir + delta_cur_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

delta_his = pd.read_csv(
    data_dir + delta_his_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

nwest_cur = pd.read_csv(
    data_dir + nwest_cur_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

nwest_his = pd.read_csv(
    data_dir + nwest_his_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

delta_cur = pd.concat([delta_cur, delta_his, nwest_cur, nwest_his])
delta_cur = delta_cur.dropna()

delta_his = None
nwest_cur = None
nwest_his = None

In [208]:
NUM = 100

x = delta_cur.Longitude[~np.isnan(delta_cur.Longitude)]
y = delta_cur.Latitude[~np.isnan(delta_cur.Latitude)]

START_X = -124
END_X = -122

START_Y = 48.2
END_Y = 49.7

xedges = np.linspace(START_X, END_X, num=NUM)
yedges = np.linspace(START_Y, END_Y,num=NUM)

dx = (END_X - START_X)/NUM
dy = (END_Y - START_Y)/NUM

H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges), normed=True)

H = H.T  # Let each row list bins with common y range.

H = sepfir2d(H, np.ones(4), np.ones(4))

#H[H>0] = 1

# Point -> (Int, Int) or False
# get i,j indices in the feasible set from coordinates
def pos2ij(point):
    if point[0] < START_X or point[0] > END_X:
        return False
    if point[1] < START_Y or point[1] > END_Y:
        return False
    
    return (int((point[0] - START_X)/dx), int((point[1] - START_Y)/dy))

def ij2pos(ij):
    i = ij[0]
    j = ij[1]
    return (i*dx + START_X, j*dy + START_Y)

In [209]:
N = NUM
M = H

def safeadd(todo,wle):
    p = wle[0]
    if p[0] >= 0 and p[0] < N - 1 and p[1] >= 0 and p[1] < N - 1:
        if M[p[0],p[1]] > 0:
            todo.append(wle)

def generatePath(p0, p1):
    # WLE is (Point, (Point))
    todo = [(p0, ())]
    while todo:
        p, path = todo.pop()
        if p == p1:
            return path
        if p not in path:
            safeadd(todo, ((p[0]+1,p[1]), (p, *path)))
            safeadd(todo, ((p[0]-1,p[1]), (p, *path)))
            safeadd(todo, ((p[0],p[1]+1), (p, *path)))
            safeadd(todo, ((p[0],p[1]-1), (p, *path)))
            todo.sort(key=lambda r: -math.hypot(r[0][0] - p1[0], r[0][1] - p1[1]))
    return []

In [ ]:
# Now let's do a ship

UserID_vc = pd.value_counts(delta_cur.UserID)
#i = random.randint(1,int(UserID_vc.size)) # 117
#i = 182 # 95 # 59
#i = 117
#i = 272
i = 206
ship = (delta_cur.loc[delta_cur.UserID == UserID_vc.index[i]])

x = list(ship.Longitude)
y = list(ship.Latitude)

p = [pos2ij(p) for p in list(zip(x,y)) if pos2ij(p)]
i = 0

while i < len(p)-1:
    if p[i] == p[i+1]:
        del p[i]
    else:
        i = i+1

p_generated = []
for idx in range(1, len(p)-1):
    if True:#(idx < len(p)*0.91):
        p_next = p[idx+1]
        p_current = p[idx]
        length = math.hypot(p_next[0] - p_current[0], p_next[1] - p_current[1])
        #if length > 2:
        p_generated.extend(generatePath(p_current, p_next))

In [ ]:
fig = plt.figure(figsize=(15, 15), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111, xlabel="Longitude", ylabel="Latitude", title="Feasible Set Of Ships")
K = np.copy(H)
K[K >0] = 1
plt.imshow(K, interpolation='nearest', origin='low',
           extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
           cmap=mpl.cm.Oranges)

p_generated_n = [ij2pos((t[0], t[1])) for t in p_generated]
(X,Y) = list(zip(*p_generated_n))
plt.scatter(X,Y, marker='*', color='g', s=50)

p_global = [ij2pos(t) for t in p]

(X,Y) = list(zip(*p_global))

plt.scatter(X,Y, marker='*', color='y', s=25)
plt.scatter(x,y, marker='.', color='r', s=2)